In [ ]:
%%capture
%pip install accelerate peft bitsandbytes trl peft tensorboard transformers==4.38.2

In [ ]:
!pip install git+https://github.com/huggingface/huggingface_hub.git@main
from huggingface_hub import login

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
login(token=os.environ['HF_TOKEN'])

In [ ]:
model_name = 'NousResearch/Meta-Llama-3-8B' # The model to fine-tune, 'NousResearch/Meta-Llama-3-8B', 'NousResearch/Llama-2-7b-hf', 'NousResearch/Llama-2-7b-chat-hf', 'mistralai/Mistral-7B-Instruct-v0.2'
prompt_format = 'llama' # or mistral
dataset_name_unlabeled = "unlabeled_data.json" # Unlabeld dataset
dataset_name_labeled = "labeled_data.json" # The instruction dataset to use
new_model_base = 'base-model' # Fine-tuned model name
new_model_semi = 'semi-supervised-model' # Semi-supervised model name

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, 'float16')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

## Training Unlabeled Data

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config, # Uncomment to use QLoRA
    device_map='auto',
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
model = prepare_model_for_kbit_training(model)
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"],
    modules_to_save=["embed_tokens", "lm_head"],
)
model = get_peft_model(model, peft_config)

In [ ]:
def generate_and_tokenize_prompt(prompt):
    return tokenizer(prompt['text'])

In [ ]:
data = load_dataset("json", data_files=dataset_name_unlabeled)

In [ ]:
dataset = data['train'].map(generate_and_tokenize_prompt)
dataset = dataset.shuffle()

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    save_steps=100000,
    logging_steps=1000,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.05,
    group_by_length=True,
    lr_scheduler_type='constant',
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model_base)

## Training Labeled Data

In [ ]:
model = PeftModel.from_pretrained(model, new_model_base)
model = model.merge_and_unload()
model = prepare_model_for_kbit_training(model)
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"],
    modules_to_save=["embed_tokens", "lm_head"],
)
model = get_peft_model(model, peft_config)

In [ ]:
data = load_dataset("json", data_files=dataset_name_labeled)
dataset = data['train'].map(generate_and_tokenize_prompt)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    save_steps=100000,
    logging_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.05,
    group_by_length=True,
    lr_scheduler_type='constant',
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model_semi)

## testing

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #quantization_config=bnb_config, # Uncomment to use QLoRA
    device_map='auto',
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = PeftModel.from_pretrained(model, new_model_semi)
model = model.merge_and_unload()

In [ ]:
import re

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

def generate(prompt):
    # format the prompt
    prompt = format_prompt(prompt)
    result = pipe(
        prompt,
        do_sample=True,
        temperature=0.9,
        top_p=0.8,
        top_k=10,
        max_new_tokens=64,
        num_return_sequences=1,
        no_repeat_ngram_size=3,
    )
    
    return format_output(prompt, result[0]['generated_text'])
    
def format_output(input, output):
    # remove the input from the output
    output = output.replace(input, '')
    # remove the punctuation that is unnecessary
    output = output.replace(':', ',').replace(';', ',')
    output = re.sub(r'[{}()\[\]]', '', output)
    return output.strip()

def format_prompt(prompt):
    # format the prompt for llama chatbot
    if prompt_format == 'llama':
        return '<s> ' + prompt
    # format the prompt for mistral instruct
    elif prompt_format == 'mistral':
        return f"<s>[INST] {prompt} [/INST]"